## Import libs, set paths and load params

In [1]:
import os
import numpy as np
import pandas as pd
import sys
import csv
sys.path.insert(0, "../src")
ART_NET_PATH = "../networks"

import auxilary_functions as f
from generation_algorithm import *
from copy import deepcopy
import networkx as nx
from time import sleep
import statistics
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load Yeast Tnet and select the FFL-based component

In [2]:
interaction_matrix = f.get_interaction_matrix(cfg)

%time
motifs, counter = f.motif_search(cfg, interaction_matrix, batch_size=10000)
motifs_orig = motifs["030T"]
counter

ffl_nodes = list(set(sum([list(map(int, x.split("_"))) for x in motifs_orig], [])))
interaction_matrix_ffl = np.zeros((len(ffl_nodes), len(ffl_nodes)))
for motif in motifs_orig:
    motif = f.split_motif(motif)
    motif_new = list(ffl_nodes.index(x) for x in motif)
    interaction_matrix_ffl[np.ix_(motif_new, motif_new)] = \
    interaction_matrix[np.ix_(motif, motif)]
interaction_matrix_ffl.shape, interaction_matrix_ffl.sum()

# Vertex-based motif network on FFL
motifs_network = f.build_vmn(motifs_orig, verbose=True)
V = nx.Graph(motifs_network)
nx.is_connected(V)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


100%|██████████| 5676765/5676765 [00:15<00:00, 364602.59it/s]


True

## Topology stats

In [3]:
def collect_topology_stats(growth_pace_vec, matrix_size_vec, initial_size_vec):
    network_topology_stats = {}
    #import resource
    
    for growth_pace_num in growth_pace_vec:
        for final_size in matrix_size_vec:
            for initial_size in initial_size_vec:
                
                #generate network
                artificial_matrix_ffl = generate_artificial_network(
                    interaction_matrix, motifs=motifs_orig, motifs_network=motifs_network,
                    nucleus_size=initial_size, network_size=final_size, growth_pace=growth_pace_num,
                    reference_matrix=interaction_matrix_ffl)
                
                sparsity = artificial_matrix_ffl[2]
                time_spent = artificial_matrix_ffl[1]
                artificial_matrix_ffl = artificial_matrix_ffl[0]

                art_matrix = deepcopy(artificial_matrix_ffl)

                if not os.path.exists(ART_NET_PATH):
                        os.mkdir(ART_NET_PATH)
                joblib.dump(
                        art_matrix, os.path.join(ART_NET_PATH,'_'+f"test{art_matrix.shape[0]}.gz")

                )

                #create edgelist
                D1 = nx.DiGraph(art_matrix)
                nx.write_edgelist(D1, os.path.join(ART_NET_PATH, "test.tsv"), data=False)

                #analyze network
                network_test = f.read_df_as_network(os.path.join(ART_NET_PATH, "test.tsv"))
                diameter = nx.diameter(network_test)
                shortest_path = nx.average_shortest_path_length(network_test)
                
                #collect stats
                key_to_dic = '_'.join(str(x) for x in [final_size,initial_size,growth_pace_num])
                list_with_stats = [growth_pace_num, final_size, initial_size, sparsity,
                                   np.round(shortest_path, 4), diameter, time_spent]
                network_topology_stats[key_to_dic] = list_with_stats

    return network_topology_stats

In [4]:
dict_with_stats = collect_topology_stats([0.6],[100],[20,25,30])

Nucleus matrix shape: (20, 20)



Network has been successfully generated!
Total time spent: 0:00:15.304498
Nucleus matrix shape: (25, 25)



Network has been successfully generated!
Total time spent: 0:00:20.103973
Nucleus matrix shape: (30, 30)



Network has been successfully generated!
Total time spent: 0:00:14.492981


In [5]:
list_with_stats = list(dict_with_stats.values())
csv_columns = ['growth_pace','size','nucleus_size', 'sparsity', 'shortest_path', 'diameter', 'time_spent']
list_with_stats.insert(0,csv_columns)

with open("../networks/topology_stats.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(list_with_stats)

In [6]:
topology_data = pd.read_csv("topology_stats.csv",header=0, delimiter=",")
topology_data

FileNotFoundError: [Errno 2] No such file or directory: 'topology_stats.csv'

In [ ]:
topology_data['shortest_path'].plot(kind='bar')

## Visualize network

In [ ]:
def read_df_as_network(filename):
    with open(filename, 'rt') as f:
        network_df = pd.read_csv(f, sep=' ', header = None)
        network = nx.from_pandas_edgelist(network_df, source = 0, target = 1)
        return network
    
network_test = read_df_as_network(os.path.join(ART_NET_PATH, "test.tsv"))

### Network

In [ ]:
nx.draw(network_test, node_size = 5, edge_color = 'gray')

### Minimum spanning tree

In [ ]:
tree = nx.minimum_spanning_tree(network_test)
nx.draw(tree, node_size = 5, edge_color = 'gray')

In [ ]:
nx.diameter(network_test)

In [ ]:
nx.average_shortest_path_length(network_test)

In [ ]:
degrees = [network_test.degree(n) for n in network_test.nodes()]
print(statistics.median(degrees))

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
network_test = [d for n, d in network_test.degree]
plt.hist(network_test, bins=100, alpha=0.75)